<a href="https://colab.research.google.com/github/nomelhorsentido/app/blob/main/io.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Install required libraries
!pip install -q diffusers transformers torch pillow gspread google-auth google-auth-oauthlib google-auth-httplib2 opencv-python-headless sentence-transformers

# Import necessary libraries
from diffusers import StableDiffusionXLInpaintPipeline
import torch
from PIL import Image
import cv2
import numpy as np
from google.colab import auth
import os
import gspread
from google.auth import default
from google.colab import drive
from sentence_transformers import SentenceTransformer, util

# Authenticate and mount Google Drive
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

# Force remount Google Drive
if os.path.exists('/content/drive'):
    drive.flush_and_unmount()
    print('Google Drive unmounted.')
drive.mount('/content/drive', force_remount=True)
print('Google Drive mounted successfully!')

# Define paths
INPUT_DIR = os.path.join('/content/drive/My Drive/input-logos')
OUTPUT_DIR = os.path.join('/content/drive/My Drive/output-logos')
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Check input directory
if not os.path.exists(INPUT_DIR):
    print(f"Error: Input directory '{INPUT_DIR}' not found.")
    os.makedirs(INPUT_DIR, exist_ok=True)
    print(f"Created input directory. Add images and restart.")
else:
    # Initialize SDXL pipeline
    device = "cuda" if torch.cuda.is_available() else "cpu"
    pipe = StableDiffusionXLInpaintPipeline.from_pretrained(
        "stabilityai/stable-diffusion-xl-base-1.0",
        torch_dtype=torch.float16,
        variant="fp16",
        use_safetensors=True
    ).to(device)

    # Initialize Sentence Transformer model
    sentence_model = SentenceTransformer('all-MiniLM-L6-v2')

    # Aesthetic configuration
    BASE_PROMPT = (
        "Abstract 3D design with strong figure-background contrast, "
        "minimal elements teaching volume and movement, "
        "vivid colors with high saturation, modern and clean composition"
    )

    def generate_abstract_prompt(image_description):
        """Generate an abstract prompt based on image description using sentence-transformers"""
        embeddings = sentence_model.encode([image_description, BASE_PROMPT])
        similarity = util.cos_sim(embeddings[0], embeddings[1])
        if similarity > 0.7:
            return f"{BASE_PROMPT} with elements inspired by {image_description}"
        else:
            return BASE_PROMPT

    def apply_lens_effect(image, distortion=0.3):
        """Applies barrel distortion for lens effect using OpenCV"""
        img = np.array(image)
        h, w = img.shape[:2]

        # Generate distortion maps
        map_x = np.zeros((h, w), np.float32)
        map_y = np.zeros((h, w), np.float32)
        scale = 1.5  # Curvature intensity

        for y in range(h):
            for x in range(w):
                nx = 2*(x - w/2)/w
                ny = 2*(y - h/2)/h
                r = np.sqrt(nx**2 + ny**2)
                theta = 1.0 / (1.0 + distortion*r**2)

                map_x[y, x] = (theta * nx * scale + 1) * w/2
                map_y[y, x] = (theta * ny * scale + 1) * h/2

        return Image.fromarray(cv2.remap(img, map_x, map_y, cv2.INTER_CUBIC))

    def process_logos(input_dir, output_dir):
        for filename in os.listdir(input_dir):
            if filename.lower().endswith((".png", ".jpg", ".jpeg")):
                input_path = os.path.join(input_dir, filename)
                try:
                    print(f"Processing: {filename}")
                    init_image = Image.open(input_path).convert("RGB").resize((1024, 1024))
                    mask_image = Image.new("RGB", init_image.size, "white")

                    # Generate image description (placeholder for actual description generation)
                    image_description = "Abstract design with strong contrast and 3D elements"

                    for i in range(4):
                        variation_prompt = generate_abstract_prompt(image_description)
                        result = pipe(
                            prompt=variation_prompt,
                            image=init_image,
                            mask_image=mask_image,
                            num_inference_steps=40,
                            strength=0.85,
                            guidance_scale=9.0
                        ).images[0]

                        # Apply lens effect
                        curved_result = apply_lens_effect(result.resize((2048, 2048)))

                        # Save output
                        output_filename = f"{os.path.splitext(filename)[0]}_variant_{i+1}.png"
                        output_path = os.path.join(output_dir, output_filename)
                        curved_result.save(output_path)
                        print(f"Saved: {output_path}")

                except Exception as e:
                    print(f"Error processing {filename}: {str(e)}")

    # Start processing
    process_logos(INPUT_DIR, OUTPUT_DIR)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 113.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 86.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 54.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 96.4 MB/s eta 0:00:00


The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

Mounted at /content/drive
Google Drive mounted successfully!


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


...

Saved: /content/drive/My Drive/output-logos/emodiversa - cortar coracao, 90graus base_variant_2.png


  0%|          | 0/34 [00:00<?, ?it/s]

Saved: /content/drive/My Drive/output-logos/emodiversa - cortar coracao, 90graus base_variant_3.png


  0%|          | 0/34 [00:00<?, ?it/s]

Saved: /content/drive/My Drive/output-logos/emodiversa - cortar coracao, 90graus base_variant_4.png
